Os incêndios florestais são um grave problema para a preservação das Florestas Tropicais. Compreender a frequência dos incêndios florestais em uma série temporal pode ajudar a tomar medidas para evitá-los.

Objetivo: Realizar a previsão de frequências de focos de incêndio para X dias.